In [1]:
import os
import sys

import logging

from autoMatch.utils.EDA import *


logging.getLogger("snowflake").setLevel(logging.WARNING)
logging.getLogger("snowflake.connector").setLevel(logging.WARNING)
logging.getLogger("snowflake.snowpark").setLevel(logging.WARNING)


%pwd
os.chdir("../")

# Add the absolute path to src/ so Python can find automatch
src_path = os.path.abspath("src")
if src_path not in sys.path:
    sys.path.append(src_path)
    
%pwd


c:\Users\fiscarelli\Desktop\Progetti\Manpower IT\Auto-Match\Candidates-to-Jobs-Auto-Match-Cortex-AI\venv\lib\site-packages\holoviews\__init__.py:81: FutureWarning: param.version.Version has been deprecated and will be removed in a future version.
  __version__ = str(param.version.Version(fpath=__file__, archive_commit="$Format:%h$",
c:\Users\fiscarelli\Desktop\Progetti\Manpower IT\Auto-Match\Candidates-to-Jobs-Auto-Match-Cortex-AI\venv\lib\site-packages\param\version.py:200: FutureWarning: param.version.run_cmd has been deprecated and will be removed in a future version.
  remotes = run_cmd([cmd, 'remote', '-v'],
c:\Users\fiscarelli\Desktop\Progetti\Manpower IT\Auto-Match\Candidates-to-Jobs-Auto-Match-Cortex-AI\venv\lib\site-packages\panel\config.py:35: FutureWarning: param.version.Version has been deprecated and will be removed in a future version.
  __version__ = str(param.version.Version(
c:\Users\fiscarelli\Desktop\Progetti\Manpower IT\Auto-Match\Candidates-to-Jobs-Auto-Match-Corte

'c:\\Users\\fiscarelli\\Desktop\\Progetti\\Manpower IT\\Auto-Match\\Candidates-to-Jobs-Auto-Match-Cortex-AI'

In [2]:
from autoMatch.utils.snowflake_utils import get_snowpark_session
session = get_snowpark_session()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/e2ba81b8-03fe-407c-96a1-f4bc0f512e7d/saml2?SAMLRequest=nZJNU9swEIb%2Fikc925IcNx%2BaJIxDYJIWSsBOmelNsddBgyy5koxJf30Vh8zQAxx688jPrh7tu9OL11oGL2Cs0GqGaERQAKrQpVD7Gdrm1%2BEYBdZxVXKpFczQASy6mE8tr2XD0tY9qQf43YJ1gW%2BkLOt%2FzFBrFNPcCssUr8EyV7Asvb1hcURYY7TThZboXcnnFdxaMM4bnktKK7zek3MNw7jruqgbRNrscUwIwWSCPXVEvpz5V%2F%2BmD3iKSXLkPeHxzZvbQqjTCD7T2p0gy1Z5vgk3d1mOgvSseqmVbWswGZgXUcD24eYkYL3BIpskJB5HnZ9bCK3RDUT8T2sgskp3leTPUOi6aZ3vHvkvXEGJpd4LP4D1coaaZ1GuhCBXjdik%2B%2B39XeLa0WF32z2SVbmQWbK6yvm3%2BzYV8ej7Y1Gg4Oc54fiY8NraFtbqmKvzRyQehoSGdJDThCUxG4wiOhj%2BQsHS%2BwnFXV95lu89oloURltdOa2kUNBbQrzjY7obh2RQQZiQURFOhpyGVbIrSPWVxjAq8THtGJ02iPUiZv6%2Fc5ni913elvKHz2m93GgpikNwrU3N3ccx0oj2J6IMqx5lUHMh07I0YK2PU0rdXRrgzu%2B%2BMy0gPD%2Fd%2

In [ ]:
df = session.sql(f"""SELECT
                    candidateid, CAST(date_available AS DATE) AS date_available, salary_low, salary, will_relocate, desired_locations, parttime_preferenza_perc, turno_preferenza
                 FROM IT_DISCOVERY.CONSUMER_INT_MODEL.CANDIDATE_CLEANED
                 --WHERE date_added >= CURRENT_DATE - INTERVAL '30 DAY'
                 WHERE turno_preferenza is not null
                 """).to_pandas()


In [ ]:
from snowflake.snowpark import functions as F

dff = session.sql(f"""SELECT
                    candidateid, CAST(date_available AS DATE) AS date_available, salary_low, will_relocate, desired_locations, parttime_preferenza_perc, turno_preferenza
                 FROM IT_DISCOVERY.CONSUMER_INT_MODEL.MPG_IT_AUTOMATCH_CANDIDATE_FEATURES_NEW
                 --WHERE date_added >= CURRENT_DATE - INTERVAL '30 DAY'
                 WHERE PARTTIME_PREFERENZA_PERC is not null
                 """)


In [37]:
bool(None)

False

In [36]:
dff.show(n=100)

---------------------------------------------------------------------------------------------------------------------------------------------
|"CANDIDATEID"  |"DATE_AVAILABLE"  |"SALARY_LOW"  |"WILL_RELOCATE"  |"DESIRED_LOCATIONS"  |"PARTTIME_PREFERENZA_PERC"  |"TURNO_PREFERENZA"  |
---------------------------------------------------------------------------------------------------------------------------------------------
|5812585        |2026-01-19        |NULL          |False            |Locale               |100                         |[                   |
|               |                  |              |                 |                     |                            |  "Mattina",        |
|               |                  |              |                 |                     |                            |  "Pomeriggio",     |
|               |                  |              |                 |                     |                            |  "Notte",          |
|     

In [93]:
from snowflake.snowpark.functions import col, regexp_replace, split, array_max, coalesce, lit, when

# Transform the dataframe
df_transformed = dff.with_column(
    "MAX_NUMBER",
    # Apply regexp_replace to clean the values, and then split by commas
    array_max(
        # Split after replacing non-numeric characters with commas
        # Ensure the split values are cast to numeric type (e.g., '10' -> 10)
        split(
            regexp_replace(coalesce(col("PARTTIME_PREFERENZA_PERC"), lit("1000")), r'[^0-9,]', lit(',')),
            lit(',')
        )
    ).cast("int")  # Cast the max value as an integer after finding the max
)

"""
df_transformed = df_transformed.with_column(
    "MAX_NUMBER",
    when(col("MAX_NUMBER") == 1000, None).otherwise(col("MAX_NUMBER"))
)
"""
df_transformed.show(n=100)


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CANDIDATEID"  |"DATE_AVAILABLE"  |"SALARY_LOW"  |"SALARY"    |"WILL_RELOCATE"  |"DESIRED_LOCATIONS"  |"PARTTIME_PREFERENZA_PERC"  |"TURNO_PREFERENZA"                       |"MAX_NUMBER"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|3865416        |2023-10-21        |NULL          |0.0000      |True             |Locale               |20%                         |Mattina,Pomeriggio,Notte                 |20            |
|4694921        |2025-03-01        |NULL          |0.0000      |False            |NULL                 |100%                        |Mattina,Pomeriggio,Notte,Fine Settimana  |100           |
|3750889        |2023-02-09        |NULL     

In [ ]:
from snowflake.snowpark.functions import col, coalesce, lit, when, greatest

ordered_values = ['0','1','10','20','30','40','50','60','70','80','90','100']

# Transform the dataframe
df_transformed = dff.with_column(
    "PARTTIME_PREFERENZA_PERC_T",
    coalesce(col("PARTTIME_PREFERENZA_PERC"), lit("1%"))
)


# Build expressions: if STRING_COL contains 'v', return int(v), else 0
exprs = [
    when(col("PARTTIME_PREFERENZA_PERC_T").contains(lit(v)), lit(int(v))).otherwise(lit(0))
    for v in ordered_values
]

# Take the maximum across all expressions
df_transformed = df_transformed.with_column(
    "PARTTIME_PREFERENZA_PERC_T",
    greatest(*exprs)
)


df_transformed = df_transformed.with_column(
    "PARTTIME_PREFERENZA_PERC_T",
    when(col("PARTTIME_PREFERENZA_PERC_T") == 1, None).otherwise(col("PARTTIME_PREFERENZA_PERC_T"))
)


df_transformed.show(n=100)


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CANDIDATEID"  |"DATE_AVAILABLE"  |"SALARY_LOW"  |"SALARY"    |"WILL_RELOCATE"  |"DESIRED_LOCATIONS"  |"PARTTIME_PREFERENZA_PERC"  |"TURNO_PREFERENZA"                       |"PARTTIME_PREFERENZA_PERC_T"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|3963958        |NULL              |NULL          |0.0000      |False            |NULL                 |NULL                        |Mattina,Pomeriggio                       |NULL                          |
|4022197        |2023-05-01        |NULL          |0.0000      |False            |Locale               |50%                         |Mattina,Pomeriggio                     

In [104]:
from snowflake.snowpark import functions as F

# Ordered list defining the hierarchy
levels = ['Locale', 'Regionale', 'Nazionale', 'Internazionale']

case_parts = []
for level in reversed(levels):   # highest first
    case_parts.append(f"WHEN DESIRED_LOCATIONS ILIKE '%{level}%' THEN '{level}'")

case_expr = "CASE " + " ".join(case_parts) + " ELSE NULL END"

dff = dff.with_column(
    "DESIRED_LOCATIONS_NORMALIZED",
    F.expr(case_expr)
)


In [105]:
dff.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CANDIDATEID"  |"DATE_AVAILABLE"  |"SALARY_LOW"  |"SALARY"  |"WILL_RELOCATE"  |"DESIRED_LOCATIONS"  |"PARTTIME_PREFERENZA_PERC"  |"TURNO_PREFERENZA"        |"DESIRED_LOCATIONS_NORMALIZED"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2540359        |2027-01-01        |NULL          |0.0000    |True             |NULL                 |50%                         |Mattina                   |NULL                            |
|4019364        |2023-11-07        |NULL          |0.0000    |False            |Locale               |100%                        |Mattina,Pomeriggio        |Locale                          |
|4391785        |2050-01-01        |2200

In [ ]:
dfcl = session.sql(f"""SELECT
                    candidateid, desired_locations,turno_preferenza
                 FROM IT_DISCOVERY.CONSUMER_INT_MODEL.MPG_IT_AUTOMATCH_CANDIDATE_FEATURES_NEW
                 --WHERE date_added >= CURRENT_DATE - INTERVAL '30 DAY'
                 WHERE  desired_locations is not null
                 """).to_pandas()

dfct = session.sql(f"""SELECT
                    candidateid, desired_locations,turno_preferenza
                 FROM IT_DISCOVERY.CONSUMER_INT_MODEL.MPG_IT_AUTOMATCH_CANDIDATE_FEATURES_NEW
                 --WHERE date_added >= CURRENT_DATE - INTERVAL '30 DAY'
                 WHERE  turno_preferenza is not null
                 """).to_pandas()



In [32]:
general_info(df)

Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CANDIDATEID               1714 non-null   int32  
 1   DATE_AVAILABLE            882 non-null    object 
 2   SALARY_LOW                240 non-null    float64
 3   SALARY                    1714 non-null   float64
 4   WILL_RELOCATE             1714 non-null   bool   
 5   DESIRED_LOCATIONS         937 non-null    object 
 6   PARTTIME_PREFERENZA_PERC  967 non-null    object 
 7   TURNO_PREFERENZA          1714 non-null   object 
dtypes: bool(1), float64(2), int32(1), object(4)
memory usage: 88.8+ KB
None


Percentage of missing values:
CANDIDATEID                  0.00
DATE_AVAILABLE              48.54
SALARY_LOW                  86.00
SALARY                       0.00
WILL_RELOCATE                0.00
DESIRED_LOCATIONS           45.33
PARTTIME_PREFERENZA_PERC

In [63]:
columns_to_check = ["WILL_RELOCATE", "DESIRED_LOCATIONS", "PARTTIME_PREFERENZA_PERC", "TURNO_PREFERENZA"]

# Print unique values for each column
for col in columns_to_check:
    print(f"Unique values in '{col}':")
    print(df[col].unique())
    print("-" * 40)



Unique values in 'WILL_RELOCATE':
[False  True]
----------------------------------------
Unique values in 'DESIRED_LOCATIONS':
['Locale' None 'Locale,Regionale' 'Regionale' 'Nazionale'
 'Regionale,Locale']
----------------------------------------
Unique values in 'PARTTIME_PREFERENZA_PERC':
['100%' None '10%' '80%' '50%' '60%' '70%' '20%' '[100%, 90%]' '30%'
 '[40%, 30%]' '[50%, 60%, 70%, 80%]' '40%' '90%']
----------------------------------------
Unique values in 'TURNO_PREFERENZA':
['Mattina,Pomeriggio' 'Mattina' 'Notte,Pomeriggio'
 'Mattina,Pomeriggio,Notte' 'Mattina,Pomeriggio,Notte,Fine Settimana'
 'Mattina,Pomeriggio,Fine Settimana' 'Fine Settimana' 'Notte'
 'Pomeriggio,Mattina,Notte' 'Notte,Mattina'
 'Pomeriggio,Mattina,Fine Settimana' 'Pomeriggio,Mattina' 'Mattina,Notte'
 'Pomeriggio,Notte' 'Pomeriggio' 'Notte,Mattina,Pomeriggio'
 'Fine Settimana,Mattina' 'Pomeriggio,Mattina,Notte,Fine Settimana'
 'Notte,Pomeriggio,Mattina' 'Pomeriggio,Notte,Fine Settimana'
 'Fine Settimana,Pom

In [ ]:
b1 = bar_plot(df, 'WILL_RELOCATE', label_range=30)
b2 = bar_plot(df, 'DESIRED_LOCATIONS', label_range=30)
b3 = bar_plot(df, 'PARTTIME_PREFERENZA_PERC', label_range=30)
b4 = bar_plot(df, 'TURNO_PREFERENZA', label_range=30)


(b1 + b2 + b3 + b4).cols(1)

:Layout
   .Bars.I   :Bars   [WILL_RELOCATE]   (COUNT)
   .Bars.II  :Bars   [DESIRED_LOCATIONS]   (COUNT)
   .Bars.III :Bars   [PARTTIME_PREFERENZA_PERC]   (COUNT)
   .Bars.IV  :Bars   [TURNO_PREFERENZA]   (COUNT)

In [ ]:
h1 = hist_plot(df, "SALARY_LOW", bins=200, xrange=[], yrange=[], height=400, width=800)
h2 = hist_plot(df, "SALARY_LOW", bins=200, xrange=[], yrange=[], height=400, width=800)


(h1 + h2).cols(1)

:Layout
   .Histogram.I  :Histogram   [SALARY_LOW]   (Frequency)
   .Histogram.II :Histogram   [SALARY_LOW]   (Frequency)

# VACANCIES

In [16]:
df = session.sql("""
                 select joborderid, requisiti, richieste_aggiuntive, dateadded, jobtitle, citta, salary, data_inizio_validita, part_time_percent, skill_list, titoli_richiesti 
                 from it_discovery.consumer_int_model.joborder_cleaned
                 where joborderid = 938151 --order by dateadded desc
            """ 
            )

In [17]:
df.select("JOBORDERID", "requisiti", "richieste_aggiuntive").show()


----------------------------------------------------------------------------------------------
|"JOBORDERID"  |"REQUISITI"                                         |"RICHIESTE_AGGIUNTIVE"  |
----------------------------------------------------------------------------------------------
|938151        |Patentino per la conduzione del carrello elevat...  |NULL                    |
|              |Esperienza nell’utilizzo del carrello retrattile    |                        |
|              |Disponibilità a lavorare su 3 turni compreso we...  |                        |
|              |Patente B                                           |                        |
----------------------------------------------------------------------------------------------



In [23]:
from autoMatch.config.configuration import ConfigurationManager
from autoMatch.components.llm import LLM
from autoMatch import logger

from autoMatch.utils.snowflake_utils import get_snowpark_session

session = get_snowpark_session()
config = ConfigurationManager()
llm_config = config.get_llm_config()
llm = LLM(config=llm_config)
response = llm.create_prompt_vacancy(session)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/e2ba81b8-03fe-407c-96a1-f4bc0f512e7d/saml2?SAMLRequest=nZJNc9owEIb%2Fikc9W5bNl9EAGRKGhDahlI9OJzchr4kGWXIlGRN%2BfYQJM%2BkhOfTmkZ9dPdp3BzfHQgYHMFZoNUQxJigAxXUm1G6INutpmKLAOqYyJrWCIXoFi25GA8sKWdJx5V7UEv5WYF3gGylLmx9DVBlFNbPCUsUKsNRxuho%2FPdIEE1oa7TTXEn0o%2BbqCWQvGecNrSWaF13txrqRRVNc1rltYm12UEEIi0o88dUa%2BXfmjf9MnfByR9pn3hMcX7263Ql1G8JXW9gJZ%2BrBeL8LFz9UaBeOr6p1WtirArMAcBIfN8vEiYL3B7arfJkmKaz%2B3ECqjS8DsVBnAVuk6l2wPXBdl5Xx37L%2BiHLJI6p3wA5hNhqjci2x3eJg%2F603dPzyx0%2Ffl7tepyIrllOyn9%2FF25n7cH8d1n8%2F3vT8pR8Hva8LJOeGZtRXM1DlX549I0g1JHMatNUlpp0U7Ce52Os8omHg%2FoZhrKq%2FyjQcuBDfa6txpJYWCxhKSLUvjbRqSVg5hm%2FR42O%2ByOMzbW07yTpxAL4vOaSfoskG0ETGj%2F53LIPrY5X0p5z6n2WShpeCvwVSbgrnPY4xx3JyILMwblELBhBxnmQFrfZxS6vrOAHN%2B952p

TypeError: sequence item 0: expected str instance, int found